# VT Graphs in Jupyter Notebook

In this notebook we will explore how to obtain attributes and relationship for different entities using VirusTotal API v3. Finally we can render all the relationships we have obtained using VTGraph.

## Import libraries

In [2]:
from msticpy.sectools.vtlookupv3 import VTLookupV3

import networkx as nx
import matplotlib.pyplot as plt
import os
import pandas as pd

try:
    import nest_asyncio
except ImportError as err:
    print("nest_asyncio is required for running VTLookup3 in notebooks.")
    resp = input("Install now? (y/n)")
    if resp.strip().lower().startswith("y"):
        !pip install nest_asyncio
        import nest_asyncio
    else:
        raise err
nest_asyncio.apply()

## Create Lookup instance

In [15]:
from msticpy.common.provider_settings import get_provider_settings
# Try to obtain key from env varaible
vt_key = os.environ.get("VT_API_KEY")
if not vt_key:
    # if not try provider settings to get from msticpyconfig.yaml
    vt_key = get_provider_settings("TIProviders")["VirusTotal"].args["AuthKey"]

In [17]:
# Instantiate vt_lookup object
vt_lookup = VTLookupV3(vt_key)

In [18]:
FILE = 'ed01ebfbc9eb5bbea545af4d01bf5f1071661840480439c6e5babe8e080e41aa'

In [19]:
example_attribute_df = vt_lookup.lookup_ioc(observable=FILE, vt_type='file')
example_attribute_df

,type_description,first_submission_date,meaningful_name,size,last_submission_date,times_submitted,detections,scans,type
id,,,,,,,,,
ed01ebfbc9eb5bbea545af4d01bf5f1071661840480439c6e5babe8e080e41aa,Win32 EXE,1494574270,diskpart.exe,3514368,1603445191,1486,67,75,file


In [20]:
example_relationship_df = vt_lookup.lookup_ioc_relationships(
    observable=FILE, 
    vt_type='file', 
    relationship='execution_parents')
example_relationship_df

target_type  \
source                                             target                                                           
ed01ebfbc9eb5bbea545af4d01bf5f1071661840480439c... 018ac8f95d5e14b92011cdbfc8c48056ca4891161ed6bdd...        file   
                                                   02a7977d1faf7bfc93a4b678a049c9495ea663e7065aa5a...        file   
                                                   06b020a3fd3296bc4c7bf53307fe7b40638e7f445bdd43f...        file   
                                                   06c676bf8f5c6af99172c1cf63a84348628ae3f39df9e52...        file   
                                                   070f603e0443b1fae57425210fb3b27c2f77d8983cfefef...        file   
...                                                                                                           ...   
                                                   f0105b922f41b0ee8595e0e7d989c9ba69d4a38337211dc...        file   
                                                   f1aa23299987eed2173e83d26b6078232051f885586ebba...        file   
                                                   f2916486e380d0c0bbd31694b05509b91f0f622478595eb...        file   
                                                   fbf74ee5d011dfb0d6c3357446ea3999ef62b088c553d66...        file   
                                                   ff6af3f113f61f823e422b7eb9e379495b81bdbb66a4e4e...        file   

                                                                                                      source_type  \
source                                             target                                                           
ed01ebfbc9eb5bbea545af4d01bf5f1071661840480439c... 018ac8f95d5e14b92011cdbfc8c48056ca4891161ed6bdd...        file   
                                                   02a7977d1faf7bfc93a4b678a049c9495ea663e7065aa5a...        file   
                                                   06b020a3fd3296bc4c7bf53307fe7b40638e7f445bdd43f...        file   
                                                   06c676bf8f5c6af99172c1cf63a84348628ae3f39df9e52...        file   
                                                   070f603e0443b1fae57425210fb3b27c2f77d8983cfefef...        file   
...                                                                                                           ...   
                                                   f0105b922f41b0ee8595e0e7d989c9ba69d4a38337211dc...        file   
                                                   f1aa23299987eed2173e83d26b6078232051f885586ebba...        file   
                                                   f2916486e380d0c0bbd31694b05509b91f0f622478595eb...        file   
                                                   fbf74ee5d011dfb0d6c3357446ea3999ef62b088c553d66...        file   
                                                   ff6af3f113f61f823e422b7eb9e379495b81bdbb66a4e4e...        file   

                                                                                                       relationship_type  
source                                             target                                                                 
ed01ebfbc9eb5bbea545af4d01bf5f1071661840480439c... 018ac8f95d5e14b92011cdbfc8c48056ca4891161ed6bdd...  execution_parents  
                                                   02a7977d1faf7bfc93a4b678a049c9495ea663e7065aa5a...  execution_parents  
                                                   06b020a3fd3296bc4c7bf53307fe7b40638e7f445bdd43f...  execution_parents  
                                                   06c676bf8f5c6af99172c1cf63a84348628ae3f39df9e52...  execution_parents  
                                                   070f603e0443b1fae57425210fb3b27c2f77d8983cfefef...  execution_parents  
...                                                                                                                  ...  
                                                   f0105b922f41b0ee8595e0e

### Obtaining result for multiple entities

The function `lookup_iocs` is able to obtain attributes for all the rows in a DataFrame. If no `observable_column` and `observable_type` parameters are specified, the function will obtain the attributes of all the entities that are in the column `target`, and will obtain their types from the `target_type` column.

This function is especially useful when a user has obtained a set of relationships, and would like to obtain their attributes.

> **Note:** it can take some time to fetch results

In [21]:
example_multiple_attribute_df = vt_lookup.lookup_iocs(example_relationship_df)
example_multiple_attribute_df

,type_description,first_submission_date,meaningful_name,size,last_submission_date,times_submitted,detections,scans,type
id,,,,,,,,,
018ac8f95d5e14b92011cdbfc8c48056ca4891161ed6bdd268770a5b56bb327f,Win32 EXE,1495139411,8479206ff1a47362199ddabebb7358d2.virus,3723264,1526215996,6,61,70,file
02a7977d1faf7bfc93a4b678a049c9495ea663e7065aa5a6caf0f69c5ff25dbd,Win32 EXE,1570020111,=?UTF-8?B?572R5piT5bel5YW3566x56uv5ZCv5YqoLmV4...,9164800,1571387079,4,53,72,file
06b020a3fd3296bc4c7bf53307fe7b40638e7f445bdd43fac1d04547a429fdaf,Win32 EXE,1588342161,Tender.pdf.exe,3991221,1588342161,1,42,74,file
06c676bf8f5c6af99172c1cf63a84348628ae3f39df9e523c42447e2045e00ff,Win32 EXE,1595479073,car.exe,4535704,1595479073,1,40,75,file
070f603e0443b1fae57425210fb3b27c2f77d8983cfefefb0ee185de572df33d,Win32 EXE,1504687270,lhdfrgui.exe,3723264,1601363298,9,64,75,file
...,...,...,...,...,...,...,...,...,...
f0105b922f41b0ee8595e0e7d989c9ba69d4a38337211dcbe86e0bdce346853d,Win32 EXE,1494880301,lhdfrgui.exe,3723264,1571667977,9,66,71,file
f1aa23299987eed2173e83d26b6078232051f885586ebba35699143b83bc68ad,Win32 EXE,1563994865,lhdfrgui.exe,3723392,1563994865,1,64,72,file
f2916486e380d0c0bbd31694b05509b91f0f622478595eba89b30031f9f64c3c,Win32 EXE,1498115823,acdsee.ultimate.10.x.unipatch_WannaCry.exe,3676610,1518624409,5,54,69,file


In [22]:
# Expand dates
example_multiple_attribute_df.assign(
    first_submission=pd.to_datetime(example_multiple_attribute_df.first_submission_date, unit="s", utc=True),
    last_submission=pd.to_datetime(example_multiple_attribute_df.last_submission_date, unit="s", utc=True)
)

,type_description,first_submission_date,meaningful_name,size,last_submission_date,times_submitted,detections,scans,type,first_submission,last_submission
id,,,,,,,,,,,
018ac8f95d5e14b92011cdbfc8c48056ca4891161ed6bdd268770a5b56bb327f,Win32 EXE,1495139411,8479206ff1a47362199ddabebb7358d2.virus,3723264,1526215996,6,61,70,file,2017-05-18 20:30:11+00:00,2018-05-13 12:53:16+00:00
02a7977d1faf7bfc93a4b678a049c9495ea663e7065aa5a6caf0f69c5ff25dbd,Win32 EXE,1570020111,=?UTF-8?B?572R5piT5bel5YW3566x56uv5ZCv5YqoLmV4...,9164800,1571387079,4,53,72,file,2019-10-02 12:41:51+00:00,2019-10-18 08:24:39+00:00
06b020a3fd3296bc4c7bf53307fe7b40638e7f445bdd43fac1d04547a429fdaf,Win32 EXE,1588342161,Tender.pdf.exe,3991221,1588342161,1,42,74,file,2020-05-01 14:09:21+00:00,2020-05-01 14:09:21+00:00
06c676bf8f5c6af99172c1cf63a84348628ae3f39df9e523c42447e2045e00ff,Win32 EXE,1595479073,car.exe,4535704,1595479073,1,40,75,file,2020-07-23 04:37:53+00:00,2020-07-23 04:37:53+00:00
070f603e0443b1fae57425210fb3b27c2f77d8983cfefefb0ee185de572df33d,Win32 EXE,1504687270,lhdfrgui.exe,3723264,1601363298,9,64,75,file,2017-09-06 08:41:10+00:00,2020-09-29 07:08:18+00:00
...,...,...,...,...,...,...,...,...,...,...,...
f0105b922f41b0ee8595e0e7d989c9ba69d4a38337211dcbe86e0bdce346853d,Win32 EXE,1494880301,lhdfrgui.exe,3723264,1571667977,9,66,71,file,2017-05-15 20:31:41+00:00,2019-10-21 14:26:17+00:00
f1aa23299987eed2173e83d26b6078232051f885586ebba35699143b83bc68ad,Win32 EXE,1563994865,lhdfrgui.exe,3723392,1563994865,1,64,72,file,2019-07-24 19:01:05+00:00,2019-07-24 19:01:05+00:00
f2916486e380d0c0bbd31694b05509b91f0f622478595eba89b30031f9f64c3c,Win32 EXE,1498115823,acdsee.ultimate.10.x.unipatch_WannaCry.exe,3676610,1518624409,5,54,69,file,2017-06-22 07:17:03+00:00,2018-02-14 16:06:49+00:00


Also, if we would like to obtain the relationships for a set of entities, we have the function `lookup_iocs_relationships`. Here also, if no `observable_column` and `observable_type` parameters are specified, the function will obtain the relationships of all the entities that are in the column `target`, and will obtain their types from the `target_type` column.

> **Note:** it can take some time to fetch results

In [23]:
example_multiple_relationship_df = vt_lookup.lookup_iocs_relationships(example_relationship_df, 'contacted_domains')
example_multiple_relationship_df

target_type  \
source                                             target                                                          
018ac8f95d5e14b92011cdbfc8c48056ca4891161ed6bdd... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com      domain   
02a7977d1faf7bfc93a4b678a049c9495ea663e7065aa5a... fkksjobnn43.org                                        domain   
070f603e0443b1fae57425210fb3b27c2f77d8983cfefef... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com      domain   
                                                   76jdd2ir2embyv47.onion                                 domain   
                                                   xxlvbrloxvriy2c5.onion                                 domain   
...                                                                                                          ...   
f1aa23299987eed2173e83d26b6078232051f885586ebba... 76jdd2ir2embyv47.onion                                 domain   
                                                   xxlvbrloxvriy2c5.onion                                 domain   
                                                   gx7ekbenv2riucmf.onion                                 domain   
                                                   57g7spgrzlojinas.onion                                 domain   
                                                   cwwnhwhlz52maqm7.onion                                 domain   

                                                                                                     source_type  \
source                                             target                                                          
018ac8f95d5e14b92011cdbfc8c48056ca4891161ed6bdd... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com        file   
02a7977d1faf7bfc93a4b678a049c9495ea663e7065aa5a... fkksjobnn43.org                                          file   
070f603e0443b1fae57425210fb3b27c2f77d8983cfefef... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com        file   
                                                   76jdd2ir2embyv47.onion                                   file   
                                                   xxlvbrloxvriy2c5.onion                                   file   
...                                                                                                          ...   
f1aa23299987eed2173e83d26b6078232051f885586ebba... 76jdd2ir2embyv47.onion                                   file   
                                                   xxlvbrloxvriy2c5.onion                                   file   
                                                   gx7ekbenv2riucmf.onion                                   file   
                                                   57g7spgrzlojinas.onion                                   file   
                                                   cwwnhwhlz52maqm7.onion                                   file   

                                                                                                      relationship_type  
source                                             target                                                                
018ac8f95d5e14b92011cdbfc8c48056ca4891161ed6bdd... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com  contacted_domains  
02a7977d1faf7bfc93a4b678a049c9495ea663e7065aa5a... fkksjobnn43.org                                    contacted_domains  
070f603e0443b1fae57425210fb3b27c2f77d8983cfefef... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com  contacted_domains  
                                                   76jdd2ir2embyv47.onion                             contacted_domains  
                                                   xxlvbrloxvriy2c5.onion                             contacted_domains  
...                                                                                                                 ...  
f1aa23299987eed2173e83d26b6078232051f885586ebba... 76jdd2ir2embyv47.onion                             conta

## Integration with VTGraph

Once we have some DataFrames with the relationships, we are able to generate and visualize a VT Graph in our notebook. The function `create_vt_graph` accepts as input a **list of Relationship DataFrames**.

> **Note:** it can take some time to generate the graph

In [24]:
graph_id = vt_lookup.create_vt_graph(
    relationship_dfs=[example_relationship_df, example_multiple_relationship_df],
    name="My first Jupyter Notebook Graph",
    private=False,
)
graph_id

'g8dbcb27e0e014eabbf75c94f2e3a7e5b502545ef3a484525a7c416e7d827aa16'

In [28]:
vt_lookup.render_vt_graph(
    graph_id = graph_id,
    width = 900,
    height = 600
)